In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests
import pandas as pd 
import numpy as np 
import random 
import urllib.request
from bs4 import BeautifulSoup

define url, change html to tree format

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

print title

In [ ]:
soup.title.string

display the table

In [ ]:
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

convert to pandas data frane

In [ ]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

drop not assigned borough

In [ ]:
df1 = df[df.Borough != 'Not assigned']
df1.head()

combime same postal code's neighbourhood with comma

In [ ]:
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)
df2.head()

if borough not assigned, assign wiht neighbourhood

In [ ]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])
df2.head()

shape

In [ ]:
df2.shape

add lat long file

In [ ]:
latlongfile = pd.read_csv('http://cocl.us/Geospatial_data')
latlongfile.head()

merge two file

In [ ]:
df3 = pd.merge(df2,latlongfile, on = 'Postal Code')
df3.head()

import important things

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

create map

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

create another df4 so not to mess up df3

In [ ]:
df4 = df3

cluster

In [ ]:
# set number of clusters
k = 5

tornoto_grouped_clustering = df3.drop(['Neighbourhood', 'Borough', 'Postal Code'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(tornoto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
df4.insert(0, 'Cluster', kmeans.labels_)

In [ ]:
df4.head()

create the map!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood'], df3['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

the end!